In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as plt
import json
%matplotlib inline

In [3]:
def read_json(name):  ##function for reading the file and changing columnnames
    df=pd.read_json(name)
    return df.rename(columns={0:"expr",1:'intent'})

In [4]:
df_is_train=read_json('is_train.json')

In [5]:
df_is_test=read_json('is_test.json')
df_is_val=read_json('is_val.json')
df_oos_train=read_json('oos_train.json')
df_oos_test=read_json('oos_test.json')
df_oos_val=read_json('oos_val.json')


In [6]:
import re
import string 
from nltk import word_tokenize
from nltk.corpus import stopwords


def clean_data(text):  #function to remove punctuatuions, tokenize the text and lower the text and remove stopwords
    text_nopunct = ''
    text_nopunct = re.sub('['+string.punctuation+']', '', text)  #no pinctations
    tokens = word_tokenize(text_nopunct)                       #convert into tokens
    lower_tokens=[x.lower() for x in tokens]                     #lowercase the tokens
    stoplist = stopwords.words('english')
    tokens_without_sw=[word for word in lower_tokens if not word in stopwords.words()]
    text_final=' '.join(tokens_without_sw) 
    return text_final
    
    
    

In [7]:
df_is_train=df_is_train.iloc[:5000,:]   #right now training data set to only 50 classes so 100*50 rows

In [8]:
df_is_train['text_Final'] = df_is_train['expr'].apply(lambda x: clean_data(x))

In [9]:
df_is_test=df_is_test.iloc[:1500,:]
df_is_test=df_is_test.append(df_oos_train)
df_is_test

,expr,intent
0,how would you say fly in italian,translate
1,what's the spanish word for pasta,translate
2,how would they say butter in zambia,translate
3,how do you say fast in spanish,translate
4,what's the word for trees in norway,translate
...,...,...
95,what percentage of species display cold bloode...,oos
96,what does it mean to be an alpha male,oos
97,what animals have alpha males,oos
98,why do males want to be alpha,oos


In [10]:
df_is_test['text_Final'] = df_is_test['expr'].apply(lambda x: clean_data(x))

In [11]:
def make_tokens(text):
    return word_tokenize(text)


In [12]:
df_is_train['tokens']=df_is_train['text_Final'].apply(lambda x: make_tokens(x))

In [13]:
df_is_test['tokens']=df_is_test['text_Final'].apply(lambda x: make_tokens(x))

In [14]:
label_names=df_is_train['intent'].unique()
new=pd.get_dummies(df_is_train['intent'])
y_train=new[label_names].values 

In [15]:
from gensim import models

In [16]:
word2vec_path = 'GoogleNews-vectors-negative300.bin.gz'
word2vec = models.KeyedVectors.load_word2vec_format(word2vec_path, binary=True)

In [17]:
all_training_words = [word for tokens in df_is_train["tokens"] for word in tokens]
training_sentence_lengths = [len(tokens) for tokens in df_is_train["tokens"]]
TRAINING_VOCAB = sorted(list(set(all_training_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_training_words), len(TRAINING_VOCAB)))
print("Max sentence length is %s" % max(training_sentence_lengths))

19530 words total, with a vocabulary size of 2353
Max sentence length is 14


In [18]:
MAX_SEQUENCE_LENGTH=14 #keeping max lenth as 6 as maximum of the sentences complete till 12 length
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer = Tokenizer(num_words=len(TRAINING_VOCAB), lower=True, char_level=False)
tokenizer.fit_on_texts(df_is_train['text_Final'].tolist())
training_sequences = tokenizer.texts_to_sequences(df_is_train['text_Final'].tolist())
train_word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(train_word_index))
train_cnn_data = pad_sequences(training_sequences, 
                               maxlen=MAX_SEQUENCE_LENGTH)##trining data ready

Found 2353 unique tokens.


In [19]:
all_testing_words = [word for tokens in df_is_test["tokens"] for word in tokens]
testing_sentence_lengths = [len(tokens) for tokens in df_is_test["tokens"]]
TESTING_VOCAB = sorted(list(set(all_testing_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_testing_words), len(TESTING_VOCAB)))
print("Max sentence length is %s" % max(testing_sentence_lengths))

6050 words total, with a vocabulary size of 1361
Max sentence length is 12


In [20]:
#MAX_SEQUENCE_LENGTH=28
test_sequences = tokenizer.texts_to_sequences(df_is_test["text_Final"].tolist())
test_cnn_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)#test_data ready

In [21]:
EMBEDDING_DIM=300
train_embedding_weights = np.zeros((len(train_word_index)+1, 
 EMBEDDING_DIM))
for word,index in train_word_index.items():
 train_embedding_weights[index,:] = word2vec[word] if word in word2vec else np.random.rand(EMBEDDING_DIM)
print(train_embedding_weights.shape)

(2354, 300)


In [188]:
from __future__ import print_function
from tensorflow.keras.layers import Embedding

from tensorflow.keras.preprocessing.text import text_to_word_sequence
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np


from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, Dropout, Activation,Flatten,Bidirectional,GRU,LSTM
from tensorflow.keras.layers import Embedding,concatenate
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D,MaxPooling1D,GlobalAveragePooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input

In [597]:
##yoon kim model not considered

def ConvNet(embeddings, max_sequence_length, num_words, embedding_dim, labels_index, trainable=True, extra_conv=True):
    
    embedding_layer = Embedding(num_words,
                            embedding_dim,
                            weights=[embeddings],
                            input_length=max_sequence_length,
                            trainable=trainable)

    sequence_input = Input(shape=(max_sequence_length,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)

    # Yoon Kim model (https://arxiv.org/abs/1408.5882)
    convs = []
    filter_sizes = [3,4,5]

    for filter_size in filter_sizes:
        l_conv = Conv1D(filters=250, kernel_size=filter_size, activation='relu')(embedded_sequences)
        l_pool = MaxPooling1D(pool_size=3)(l_conv)
        convs.append(l_pool)

    l_merge = concatenate(convs, axis=1)

    # add a 1D convnet with global maxpooling, instead of Yoon Kim model
    #conv = Conv1D(filters=128, kernel_size=3, activation='relu')(embedded_sequences)
    #pool = MaxPooling1D(pool_size=3)(conv)

    if extra_conv==True:
        x = Dropout(0.1)(l_merge)  
    else:
        # Original Yoon Kim model
        x = Dropout(0.001)(pool)
    x = Flatten()(x)
    #x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)
    # Finally, we feed the output into a Sigmoid layer.
    # The reason why sigmoid is used is because we are trying to achieve a binary classification(1,0) 
    # for each of the 6 labels, and the sigmoid function will squash the output between the bounds of 0 and 1.
    preds = Dense(labels_index, activation='sigmoid')(x)

    model = Model(sequence_input, preds)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['acc'])
    model.summary()
    return model

In [598]:
MAX_SEQUENCE_LENGTH=14

In [599]:
x_train = train_cnn_data
y_tr = y_train

In [600]:
model = ConvNet(train_embedding_weights, MAX_SEQUENCE_LENGTH, len(train_word_index)+1, EMBEDDING_DIM, 
                len(list(label_names)), True)

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 14)]         0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 14, 300)      706200      input_2[0][0]                    
__________________________________________________________________________________________________
conv1d_5 (Conv1D)               (None, 12, 250)      225250      embedding_2[0][0]                
__________________________________________________________________________________________________
conv1d_6 (Conv1D)               (None, 11, 250)      300250      embedding_2[0][0]                
____________________________________________________________________________________________

In [601]:
#define callbacks
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=4, verbose=1)
callbacks_list = [early_stopping]

In [602]:
batch_size = 128 
num_epochs = 8
hist = model.fit(x_train, y_tr, epochs=num_epochs, callbacks=callbacks_list, validation_split=0.1, shuffle=True, batch_size=batch_size)

Train on 4500 samples, validate on 500 samples
Epoch 1/8
4500/4500 [==============================] - ETA: 31s - loss: 0.7005 - acc: 0.48 - ETA: 18s - loss: 0.6698 - acc: 0.64 - ETA: 14s - loss: 0.6392 - acc: 0.74 - ETA: 12s - loss: 0.6125 - acc: 0.80 - ETA: 11s - loss: 0.5847 - acc: 0.83 - ETA: 9s - loss: 0.5524 - acc: 0.8605 - ETA: 9s - loss: 0.5229 - acc: 0.877 - ETA: 8s - loss: 0.4936 - acc: 0.890 - ETA: 7s - loss: 0.4642 - acc: 0.900 - ETA: 7s - loss: 0.4360 - acc: 0.908 - ETA: 6s - loss: 0.4102 - acc: 0.914 - ETA: 6s - loss: 0.3865 - acc: 0.920 - ETA: 5s - loss: 0.3661 - acc: 0.924 - ETA: 5s - loss: 0.3483 - acc: 0.928 - ETA: 5s - loss: 0.3333 - acc: 0.932 - ETA: 4s - loss: 0.3202 - acc: 0.935 - ETA: 4s - loss: 0.3094 - acc: 0.937 - ETA: 4s - loss: 0.3002 - acc: 0.940 - ETA: 4s - loss: 0.2916 - acc: 0.942 - ETA: 3s - loss: 0.2843 - acc: 0.944 - ETA: 3s - loss: 0.2778 - acc: 0.945 - ETA: 3s - loss: 0.2721 - acc: 0.947 - ETA: 2s - loss: 0.2666 - acc: 0.948 - ETA: 2s - loss: 0.2615 

In [603]:
y_test = model.predict(test_cnn_data, batch_size=64, verbose=1)

1600/1600 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 1s 447us/sample


In [604]:
original_ans=df_is_test['intent'].tolist()

In [605]:
def check_metric(y_test,original_ans):
    output_class_pred=[]
    y_test=y_test.tolist()
    acc=[]
    pre=[]
    recall=[]
    f1=[]
    specificity=[]
    sensitivity=[]
    GMean1=[]
    Gmean2=[]
    MCC=[]
    for i in range(len(y_test)):
        m=max(y_test[i])
        output_class_pred.append([label_names[y_test[i].index(m)],m])
        
    for t in [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]: 
        for i in range(len(y_test)):
            if(output_class_pred[i][1]<t):    ##t is threshold
                output_class_pred[i][0]="oos"
        
        rightly_predicted_count=0
        for i in range(len(y_test)):
            if( output_class_pred[i][0]==original_ans[i]):
                rightly_predicted_count+=1

        accuracy=rightly_predicted_count/len(y_test)
        print('accuracy',rightly_predicted_count/len(y_test))
        TP=0
        FP=0
        TN=0
        FN=0
        for i in range(len(y_test)):
            if(  original_ans[i]!='oos' and output_class_pred[i][0]=='oos' ):
                FP+=1
        
            if(original_ans[i]=="oos" and output_class_pred[i][0]=='oos' ):
                TP+=1
        
            if(original_ans[i]=="oos" and output_class_pred[i][0]!='oos' ):
                FN+=1
        
            if(original_ans[i]!="oos" and output_class_pred[i][0]!='oos' ):
                TN+=1
        precision=TP/(TP+FP)
        recalll=TP/(FN+TP)
        F1=2*precision*recalll/(precision+recalll)
        sensiti=TP/(TP+FN)
        specifici=TN/(TN+FP)
        numerator=TP*TN - FP*FN
    
        denominator=np.sqrt((TP+FP)*(FN+TN)*(FP+TN)* (TP+FN))
        MCc=numerator/denominator
        G_mean1=np.sqrt(sensiti*precision)
        G_mean2=np.sqrt(sensiti*specifici)
        print('precision_oos:' ,TP/(TP+FP))
        print('recall_oos:',TP/(FN+TP))
        print("F1_oos:",F1)
        print("Specificity_oos:",TN/(TN+FP))
        print("Sensitivity_oos ",TP/(TP+FN))
        print('G-mean1:',np.sqrt(sensiti*precision))
        print("G-mean2",np.sqrt(sensiti*specifici))
        print("MCC :",MCc)
        acc.append(accuracy)
        pre.append(precision)
        recall.append(recalll)
        f1.append(F1)
        specificity.append(specifici)
        sensitivity.append(sensiti)
        GMean1.append(G_mean1)
        Gmean2.append(G_mean2)
        MCC.append(MCc)
        matrix=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
    data={'threshold':matrix,'accuracy_all':acc,"precision":pre,'recall':recall,'F1_score':f1,'specificity':specificity,'sensitivity':sensitivity,'Gmean1':GMean1,"Gmean2":Gmean2,"MCC":MCC}
    metric=pd.DataFrame(data)
    return metric
    
    
    
    
    

In [606]:
resi=check_metric(y_test,original_ans)

accuracy 0.6525
precision_oos: 0.1092436974789916
recall_oos: 0.13
F1_oos: 0.11872146118721462
Specificity_oos: 0.9293333333333333
Sensitivity_oos  0.13
G-mean1: 0.11917080461366747
G-mean2 0.3475821245883242
MCC : 0.05473855985667431
accuracy 0.62375
precision_oos: 0.13109756097560976
recall_oos: 0.43
F1_oos: 0.20093457943925236
Specificity_oos: 0.81
Sensitivity_oos  0.43
G-mean1: 0.23742778106092008
G-mean2 0.5901694671871801
MCC : 0.14390516895863842
accuracy 0.578125
precision_oos: 0.11764705882352941
recall_oos: 0.68
F1_oos: 0.20058997050147492
Specificity_oos: 0.66
Sensitivity_oos  0.68
G-mean1: 0.282842712474619
G-mean2 0.6699253689777691
MCC : 0.17133069613002574
accuracy 0.544375
precision_oos: 0.13101604278074866
recall_oos: 0.98
F1_oos: 0.2311320754716981
Specificity_oos: 0.5666666666666667
Sensitivity_oos  0.98
G-mean1: 0.358323487822294
G-mean2 0.7452069063913279
MCC : 0.2652147211237289
accuracy 0.495625
precision_oos: 0.11785714285714285
recall_oos: 0.99
F1_oos: 0.210638

In [610]:
resi

,threshold,accuracy_all,precision,recall,F1_score,specificity,sensitivity,Gmean1,Gmean2,MCC
0,0.1,0.652500,0.109244,0.13,0.118721,0.929333,0.13,0.119171,0.347582,0.054739
1,0.2,0.623750,0.131098,0.43,0.200935,0.810000,0.43,0.237428,0.590169,0.143905
2,0.3,0.578125,0.117647,0.68,0.200590,0.660000,0.68,0.282843,0.669925,0.171331
3,0.4,0.544375,0.131016,0.98,0.231132,0.566667,0.98,0.358323,0.745207,0.265215
4,0.5,0.495625,0.117857,0.99,0.210638,0.506000,0.99,0.341582,0.707771,0.240426
5,0.6,0.460000,0.107609,0.99,0.194118,0.452667,0.99,0.326393,0.669433,0.216757
6,0.7,0.405000,0.097371,1.00,0.177462,0.382000,1.00,0.312043,0.618061,0.192862
7,0.8,0.353750,0.089526,1.00,0.164339,0.322000,1.00,0.299208,0.567450,0.169786
8,0.9,0.265625,0.079051,1.00,0.146520,0.223333,1.00,0.281161,0.472582,0.132871


In [608]:
resi.to_csv('results.csv', mode='w', index = False, header=res.columns,columns=res.columns)

In [ ]:
#### the propesed model

In [572]:
embeddings=train_embedding_weights
max_sequence_length=MAX_SEQUENCE_LENGTH
num_words=len(train_word_index)+1
trainable=False
embedding_dim=EMBEDDING_DIM

In [660]:
def build_model(hp):
    model=Sequential()
    model.add(Embedding(num_words,embedding_dim,weights=[embeddings],input_length=max_sequence_length,trainable=trainable))
    #model.add(Bidirectional(GRU(150, return_sequences=True,dropout=0.1,recurrent_dropout=0.1)))
    model.add(Conv1D(
            filters=hp.Int('conv_1_filter',min_value=16,max_value=128,step=16),
            kernel_size=hp.Choice('conv_1_kernel',values=[3,5]),
            activation='relu'
        ))
    #model.add(MaxPooling1D()),
    model.add(Conv1D(
            filters=hp.Int('conv_2_filter',min_value=16,max_value=128,step=16),
            kernel_size=hp.Choice('conv_1_kernel',values=[3,5]),
            activation='relu'  
        ))
    #model.add(MaxPooling1D())
    #model.add(Conv1D(
            #filters=hp.Int('conv_2_filter',min_value=16,max_value=128,step=16),
            #kernel_size=hp.Choice('conv_1_kernel',values=[3,5]),
            #activation='relu'  
        #))
    model.add(MaxPooling1D())
    model.add(Flatten())
    model.add(Dense(
            units=hp.Int('dense_1_units',min_value=32,max_value=128,step=16),
            activation='relu'
        ))
    #model.add(Dense(
            #units=hp.Int('dense_2_units',min_value=32,max_value=256,step=16),
            #activation='relu'
        #))
    model.add(Dropout(0.1))
    #model.add(Flatten())
    model.add(Dense(units=50,activation='sigmoid'))
    #model=Model(Input(shape=(max_sequence_length,), dtype='int32'),pred)
    model.compile(optimizer=Adam(hp.Choice('learning_rate',values=[1e-2,1e-3,1e-1])),
    loss='binary_crossentropy',
    metrics=['accuracy'])
    return model

In [661]:
from kerastuner import RandomSearch
from tensorflow.keras.optimizers import Adam

In [662]:
from kerastuner.engine.hyperparameters import HyperParameters

In [663]:
tuner_search=RandomSearch(build_model,objective="val_accuracy",max_trials=5,directory='o5u4eu',project_name='open_world')

INFO:tensorflow:Reloading Oracle from existing project o5u4eu/open_world/oracle.json
INFO:tensorflow:Reloading Tuner from o5u4eu/open_world/tuner0.json


In [664]:
tuner_search.search(x_train,y_tr,epochs=3,validation_split=0.1)

INFO:tensorflow:Oracle triggered exit


In [665]:
model=tuner_search.get_best_models(num_models=1)[0]

In [666]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 14, 300)           706200    
_________________________________________________________________
conv1d (Conv1D)              (None, 12, 112)           100912    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 10, 80)            26960     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 5, 80)             0         
_________________________________________________________________
flatten (Flatten)            (None, 400)               0         
_________________________________________________________________
dense (Dense)                (None, 112)               44912     
_________________________________________________________________
dropout (Dropout)            (None, 112)               0

In [667]:
model.fit(x_train,y_tr,epochs=10,validation_split=0.1,initial_epoch=3)

Train on 4500 samples, validate on 500 samples
Epoch 4/10
4500/4500 [==============================] - ETA: 2:18 - loss: 0.0955 - accuracy: 0.98 - ETA: 23s - loss: 0.0944 - accuracy: 0.9801 - ETA: 14s - loss: 0.0934 - accuracy: 0.980 - ETA: 8s - loss: 0.0933 - accuracy: 0.980 - ETA: 7s - loss: 0.0932 - accuracy: 0.98 - ETA: 6s - loss: 0.0927 - accuracy: 0.98 - ETA: 5s - loss: 0.0919 - accuracy: 0.98 - ETA: 4s - loss: 0.0917 - accuracy: 0.98 - ETA: 3s - loss: 0.0913 - accuracy: 0.98 - ETA: 3s - loss: 0.0908 - accuracy: 0.98 - ETA: 2s - loss: 0.0902 - accuracy: 0.98 - ETA: 2s - loss: 0.0898 - accuracy: 0.98 - ETA: 2s - loss: 0.0890 - accuracy: 0.98 - ETA: 1s - loss: 0.0884 - accuracy: 0.98 - ETA: 1s - loss: 0.0876 - accuracy: 0.98 - ETA: 1s - loss: 0.0871 - accuracy: 0.98 - ETA: 1s - loss: 0.0865 - accuracy: 0.98 - ETA: 1s - loss: 0.0858 - accuracy: 0.98 - ETA: 1s - loss: 0.0849 - accuracy: 0.98 - ETA: 0s - loss: 0.0842 - accuracy: 0.98 - ETA: 0s - loss: 0.0836 - accuracy: 0.98 - ETA: 0s

4500/4500 [==============================] - ETA: 1s - loss: 0.0088 - accuracy: 0.99 - ETA: 1s - loss: 0.0059 - accuracy: 0.99 - ETA: 1s - loss: 0.0057 - accuracy: 0.99 - ETA: 1s - loss: 0.0057 - accuracy: 0.99 - ETA: 1s - loss: 0.0057 - accuracy: 0.99 - ETA: 1s - loss: 0.0059 - accuracy: 0.99 - ETA: 1s - loss: 0.0061 - accuracy: 0.99 - ETA: 1s - loss: 0.0062 - accuracy: 0.99 - ETA: 1s - loss: 0.0063 - accuracy: 0.99 - ETA: 1s - loss: 0.0062 - accuracy: 0.99 - ETA: 0s - loss: 0.0062 - accuracy: 0.99 - ETA: 0s - loss: 0.0064 - accuracy: 0.99 - ETA: 0s - loss: 0.0064 - accuracy: 0.99 - ETA: 0s - loss: 0.0064 - accuracy: 0.99 - ETA: 0s - loss: 0.0064 - accuracy: 0.99 - ETA: 0s - loss: 0.0064 - accuracy: 0.99 - ETA: 0s - loss: 0.0063 - accuracy: 0.99 - ETA: 0s - loss: 0.0062 - accuracy: 0.99 - ETA: 0s - loss: 0.0061 - accuracy: 0.99 - ETA: 0s - loss: 0.0061 - accuracy: 0.99 - ETA: 0s - loss: 0.0062 - accuracy: 0.99 - ETA: 0s - loss: 0.0061 - accuracy: 0.99 - ETA: 0s - loss: 0.0062 - accura

In [668]:
y_test= model.predict(test_cnn_data, verbose=1)

1600/1600 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - 0s 153us/sample


In [669]:
def check_metric(y_test,original_ans):
    output_class_pred=[]
    y_test=y_test.tolist()
    acc=[]
    pre=[]
    recall=[]
    f1=[]
    specificity=[]
    sensitivity=[]
    GMean1=[]
    Gmean2=[]
    MCC=[]
    for i in range(len(y_test)):
        m=max(y_test[i])
        output_class_pred.append([label_names[y_test[i].index(m)],m])
        
    for t in [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]: 
        for i in range(len(y_test)):
            if(output_class_pred[i][1]<t):    ##t is threshold
                output_class_pred[i][0]="oos"
        
        rightly_predicted_count=0
        for i in range(len(y_test)):
            if( output_class_pred[i][0]==original_ans[i]):
                rightly_predicted_count+=1

        accuracy=rightly_predicted_count/len(y_test)
        print('accuracy',rightly_predicted_count/len(y_test))
        TP=0
        FP=0
        TN=0
        FN=0
        for i in range(len(y_test)):
            if(  original_ans[i]!='oos' and output_class_pred[i][0]=='oos' ):
                FP+=1
        
            if(original_ans[i]=="oos" and output_class_pred[i][0]=='oos' ):
                TP+=1
        
            if(original_ans[i]=="oos" and output_class_pred[i][0]!='oos' ):
                FN+=1
        
            if(original_ans[i]!="oos" and output_class_pred[i][0]!='oos' ):
                TN+=1
        precision=TP/(TP+FP)
        recalll=TP/(FN+TP)
        F1=2*precision*recalll/(precision+recalll)
        sensiti=TP/(TP+FN)
        specifici=TN/(TN+FP)
        numerator=TP*TN - FP*FN
    
        denominator=np.sqrt((TP+FP)*(FN+TN)*(FP+TN)* (TP+FN))
        MCc=numerator/denominator
        G_mean1=np.sqrt(sensiti*precision)
        G_mean2=np.sqrt(sensiti*specifici)
        print('precision_oos:' ,TP/(TP+FP))
        print('recall_oos:',TP/(FN+TP))
        print("F1_oos:",F1)
        print("Specificity_oos:",TN/(TN+FP))
        print("Sensitivity_oos ",TP/(TP+FN))
        print('G-mean1:',np.sqrt(sensiti*precision))
        print("G-mean2",np.sqrt(sensiti*specifici))
        print("MCC :",MCc)
        acc.append(accuracy)
        pre.append(precision)
        recall.append(recalll)
        f1.append(F1)
        specificity.append(specifici)
        sensitivity.append(sensiti)
        GMean1.append(G_mean1)
        Gmean2.append(G_mean2)
        MCC.append(MCc)
        matrix=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
    data={'threshold':matrix,'accuracy_all':acc,"precision":pre,'recall':recall,'F1_score':f1,'specificity':specificity,'sensitivity':sensitivity,'Gmean1':GMean1,"Gmean2":Gmean2,"MCC":MCC}
    metric=pd.DataFrame(data)
    return metric
    
    
    
    
    

In [670]:
res=check_metric(y_test,original_ans)

accuracy 0.730625
precision_oos: 0.2558139534883721
recall_oos: 0.22
F1_oos: 0.23655913978494625
Specificity_oos: 0.9573333333333334
Sensitivity_oos  0.22
G-mean1: 0.2372321010475645
G-mean2 0.458926283114547
MCC : 0.19033727348336452
accuracy 0.733125
precision_oos: 0.26282051282051283
recall_oos: 0.41
F1_oos: 0.3203125
Specificity_oos: 0.9233333333333333
Sensitivity_oos  0.41
G-mean1: 0.3282627153004286
G-mean2 0.6152777150739873
MCC : 0.2720059969061589
accuracy 0.724375
precision_oos: 0.23605150214592274
recall_oos: 0.55
F1_oos: 0.3303303303303303
Specificity_oos: 0.8813333333333333
Sensitivity_oos  0.55
G-mean1: 0.36031698014423014
G-mean2 0.6962279320260955
MCC : 0.2960032185625604
accuracy 0.72375
precision_oos: 0.2422360248447205
recall_oos: 0.78
F1_oos: 0.36966824644549773
Specificity_oos: 0.8373333333333334
Sensitivity_oos  0.78
G-mean1: 0.4346770058087752
G-mean2 0.8081584003151857
MCC : 0.3727108145509367
accuracy 0.704375
precision_oos: 0.21188630490956073
recall_oos: 0.82

In [671]:
res

,threshold,accuracy_all,precision,recall,F1_score,specificity,sensitivity,Gmean1,Gmean2,MCC
0,0.1,0.730625,0.255814,0.22,0.236559,0.957333,0.22,0.237232,0.458926,0.190337
1,0.2,0.733125,0.262821,0.41,0.320312,0.923333,0.41,0.328263,0.615278,0.272006
2,0.3,0.724375,0.236052,0.55,0.330330,0.881333,0.55,0.360317,0.696228,0.296003
3,0.4,0.723750,0.242236,0.78,0.369668,0.837333,0.78,0.434677,0.808158,0.372711
4,0.5,0.704375,0.211886,0.82,0.336756,0.796667,0.82,0.416829,0.808249,0.348586
5,0.6,0.690625,0.189815,0.82,0.308271,0.766667,0.82,0.394523,0.792885,0.319870
6,0.7,0.669375,0.172131,0.84,0.285714,0.730667,0.84,0.380250,0.783428,0.300031
7,0.8,0.636875,0.152632,0.87,0.259701,0.678000,0.87,0.364403,0.768023,0.276994
8,0.9,0.586250,0.137881,0.95,0.240811,0.604000,0.95,0.361921,0.757496,0.270824


In [585]:
res.to_csv('results.csv', mode='w', index = False, header=res.columns,columns=res.columns)